# Manhattan locations analysis for targeted advertisement

Clustering of advertising kiosks in Manhattan based on data from FourSquare API

### Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

    <ol> 
        <li><a href="#Introduction/Business-Problem">Introduction/Business Problem</a></li>
        <li><a href="#Data">Data</a>
            <ul>
                <li><a href="#Data-Sources">Data Sources</a></li>
                <li><a href="#Exploratory-Data-Analysis">Exploratory Data Analysis</a></li>
            </ul>
        </li>
        <li><a href="#Methodology">Methodology</a>
            <ul>
                <li><a href="#Clustering">Clustering</a></li>
                <li><a href="#Analysis-of-our-clusters">Analysis of our clusters</a></li>
                <li><a href="#Visualization">Visualization</a></li>
            </ul>
        </li>
        <li><a href="#Results-and-Conclusion">Results and Conclusion</a>
    </ol>
</div>

## Introduction/Business Problem

Consider you as owner of some kiosks network in Manhattan. Let's  say you have the some small network of 100 locations similar to LinkNYC https://www.link.nyc/

![Kiosks](images/kiosks.png)

Similarly to LinkNYC your spots provide free Wi-Fi, device charging etc.

Yor network is completely free because it’s funded through advertising. Its groundbreaking advertising network not only provides brands with a rich, context-aware platform to reach New Yorkers and visitors, but will generate big revenue for New York City. 

You have scope of customers for your advertising platform. Most of them sell products for women. Such as Fashion, Jewelry, Shoes etc.

Your goal is to make some analysis of your spots to do better offers segmentation and price diversification for different locations. Basing on people testimonials from places nearby.

With data based on recommended and popular venues from FourSquare. With their API.

Depending ont the analysis your company will build strategy for different types of customers and their products. Trying to reach the best price options and best location for their ads.

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import os
import re

from sklearn.cluster import KMeans
import folium
import json

In [7]:
# add env variables for your foursquare and google api keys
# more secure way

In [5]:
FOURSQUARE_API_CLIENT_ID = os.getenv("FOURSQUARE_API_CLIENT_ID")
FOURSQUARE_API_CLIENT_SECRET = os.getenv("FOURSQUARE_API_CLIENT_SECRET")
FOURSQUARE_API_VERSION = '20180605' # Foursquare API version
FOURSQUARE_API_LIMIT=100

In [6]:
address = 'Manhattan, New York, US'

location = geocoder.google(address, key=os.getenv("GOOGLE_API_KEY")).latlng
latitude = location[0]
longitude = location[1]
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Manhattan, New York, US are 40.7830603, -73.9712488.


In [8]:
# Foursquare API query wiith caching option 
# dealing with free API developer plan :)

def fs_api_query(url):
    
    url_hash = url.__hash__()
    
    cache_file = "cache/" + str(url_hash) + ".json"
    
    result = ""
    
    if os.path.isfile(cache_file):
        with open(cache_file) as fp:
            result = json.load(fp)
    else:
        result = requests.get(url).json()
        with open(cache_file, "w") as fp:
            json.dump(result, fp)
          
    return result


In [9]:
def convert_foursquare_json_to_pandadf(json_data, categories_filter="Restaurant|Noodle|Sake Bar"):
    
    if "groups" in json_data["response"]:
        items = [v['venue'] for v in json_data["response"]["groups"][0]["items"]]
    else:
        items = [v for v in json_data["response"]["venues"]]
    
    venues = []
    
    venues.append([(
            v['id'], 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name'] if len(v['categories']) else np.nan,
            v['location']['postalCode'] if 'postalCode' in v['location'] else np.nan,
            v['location']['address'] if 'address' in v['location'] else np.nan, 
            v['location']['distance']
        ) for v in items])
        
    df = pd.DataFrame([item for venues in venues for item in venues])
    df.columns = [
                'Id',
                'Name', 
                'Latitude', 
                'Longitude',  
                'Category',
                'PostalCode',
                'Address',
                'Distance'
    ]
    df["Street"] = df["Address"].str.replace("^\d+\s", "")
    df["Category"] = df["Category"].fillna("")
    if categories_filter:
        df = df[df["Category"].str.contains(categories_filter)]
    return df


In [10]:
def get_nearby_venues(location_data, radius=500, categories_filter=False):
    
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}'\
        '&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
    .format(FOURSQUARE_API_CLIENT_ID, FOURSQUARE_API_CLIENT_SECRET, 
            location_data["Latitude"], location_data["Longitude"], 
            FOURSQUARE_API_VERSION, radius, FOURSQUARE_API_LIMIT)
    
    venues_json = fs_api_query(url)
    df = convert_foursquare_json_to_pandadf(venues_json, categories_filter=categories_filter)
    
    uniq_col_name = "LocationName"
    df[uniq_col_name] = location_data["Street Address"]
    # move important column to the first column
    cols = list(df.columns)
    cols.remove(uniq_col_name)
    reordered_cols = [uniq_col_name] + cols
    return df[reordered_cols]
    

## Data 
Data sources and its exploratory analysis.

### Data Sources

In our work we are going to use two data sources:

1) <b>LinkNYC kiosks locations dataset from https://data.cityofnewyork.us</b>

https://data.cityofnewyork.us/Social-Services/LinkNYC-Locations/s4kf-3yrf

LinkNYC's guys are generous and smart. And posts their locations dataset to NYC Open Data project. 

In our work we will gently use their locations as our sample kiosks dataset.



2) <b>Foursquare Places API</b>

https://developer.foursquare.com/places-api

The main data source giving us information on popular places, people testimonials and other useful data neaarby our kiosks.

In the API the method we will use is "explore" https://developer.foursquare.com/docs/api/venues/explore. That "Returns a list of recommended venues near the current location". 

For different purposes in our analysis we may use various endpoints such as:

https://developer.foursquare.com/docs/api/venues/trending Returns a list of venues near the current location with the most people currently checked in. May be useful for long-term time based analysis for your strategies

https://developer.foursquare.com/docs/api/venues/suggestcompletion Returns a list of mini-venues partially matching the search term, near the location. 

We could use various methods to get venues data. But our goal is to target products to popular locations that could be provided with "explore" method. That's how we can explain our choice of API endpoint.

<b>Additional API using</b>

<b>Nearby Places categories</b> taken from FourSquare categories
https://developer.foursquare.com/docs/resources/categories

### Exploratory Data Analysis

#### Getting sample kiosks location

LinkNYC kiosks locations
from https://data.cityofnewyork.us/Social-Services/LinkNYC-Locations/s4kf-3yrf dataset

In [11]:
# LinkNYC kiosks locations
# from https://data.cityofnewyork.us/Social-Services/LinkNYC-Locations/s4kf-3yrf dataset
kiosks = pd.read_csv("https://data.cityofnewyork.us/api/views/s4kf-3yrf/rows.csv?accessType=DOWNLOAD")

In [12]:
kiosks.describe()

,Community Board,Council District,Latitude,Longitude,Smallest PPT,Postcode,Building Identification Number (BIN),Borough-Block-Lot (BBL),Census Tract (CT)
count,1867.000000,1867.000000,1867.000000,1867.000000,1865.000000,1867.000000,1.867000e+03,1.867000e+03,1.867000e+03
mean,190.480986,13.457954,40.754821,-73.956578,129590.294906,10402.903589,1.910738e+06,1.864858e+09,1.865630e+06
std,120.604355,13.113068,0.052789,0.044966,11325.474096,544.109195,1.246825e+06,1.221483e+09,1.220849e+06
min,101.000000,1.000000,40.561755,-74.116075,100190.000000,10001.000000,1.000000e+06,1.000328e+09,1.000700e+06
25%,106.000000,4.000000,40.731158,-73.983325,121312.000000,10019.000000,1.033308e+06,1.010350e+09,1.010100e+06
50%,109.000000,7.000000,40.753746,-73.966224,126532.000000,10030.000000,1.063962e+06,1.018690e+09,1.019100e+06
75%,302.000000,25.000000,40.790567,-73.939069,138316.000000,11103.000000,3.032305e+06,3.011795e+09,3.020100e+06
max,503.000000,50.000000,40.868698,-73.792205,146100.000000,11435.000000,5.158060e+06,5.039830e+09,5.018100e+06


#### Filtering kiosks by Manhattan borough

And getting 100 random kiosks. We are small company yet :) No worries now, Jeff Bezos.

In [13]:
manhattan_kiosks = kiosks[kiosks["Borough"] == "Manhattan"]

# say we have not more than 100 kiosk
# choosing random kiosks
manhattan_kiosks = manhattan_kiosks.sample(100, random_state=0)

In [14]:
manhattan_kiosks.head()

,CB Link ID,Borough,Community Board,Council District,Latitude,Longitude,Project Status,Smallest PPT,Street Address,Postcode,Link Site ID,Link Smoke Tested and Activated (A),Link Installation (A),Neighborhood Tabulation Area (NTA),Building Identification Number (BIN),Borough-Block-Lot (BBL),Census Tract (CT),Location
528,LINK-003308,Manhattan,105,3,40.758022,-73.985832,Live,133536.0,200 WEST 45 STREET,10036,mn-05-133536,05/16/2018,11/14/2017,Midtown-Midtown South,1024714,1010160036,1011900,"(40.758022, -73.985832)"
1155,LINK-011151,Manhattan,106,2,40.737373,-73.981159,Live,135679.0,384 2 AVENUE,10010,mn-06-135679,01/03/2018,09/28/2017,Gramercy,1020569,1009280001,1006400,"(40.73737306, -73.98115859)"
1615,LINK-010610,Manhattan,107,6,40.785774,-73.972463,Live,134114.0,77 WEST 85 STREET,10024,mn-07-134114,04/30/2018,03/14/2018,Upper West Side,1031349,1011990001,1016900,"(40.78577388, -73.97246279)"
1189,LINK-000502,Manhattan,105,3,40.749880,-73.994993,Live,136910.0,408 8 AVENUE,10001,mn-05-136910,06/30/2017,05/26/2017,Midtown-Midtown South,1014351,1007807501,1010100,"(40.74988, -73.994993)"
1304,LINK-010843,Manhattan,102,3,40.732308,-74.006415,Live,123611.0,482 HUDSON STREET,10014,mn-02-123611,01/18/2017,11/05/2016,West Village,1009852,1005850012,1007300,"(40.73230778, -74.00641536)"


#### Getting Foursquare venues nearby our kiosks

Once again, our goal is to target products to popular locations that could be provided with "explore" method. That's how we can explain our choice of API endpoint. https://developer.foursquare.com/docs/api/venues/explore

In [15]:
venues = get_nearby_venues(
    manhattan_kiosks.iloc[0]
)



for index, row in manhattan_kiosks.iloc[1:].iterrows():
    venues = venues.append(
        get_nearby_venues(row)
    )


venues.head()

,LocationName,Id,Name,Latitude,Longitude,Category,PostalCode,Address,Distance,Street
0,200 WEST 45 STREET,4ec0a81fb6341cd41f1ef402,Minskoff Theatre,40.757494,-73.985680,Theater,10036,1515 Broadway,60,Broadway
1,200 WEST 45 STREET,59580ce6db1d8148fee3d383,Los Tacos No. 1,40.757134,-73.987536,Taco Place,10036,229 W 43rd St,174,W 43rd St
2,200 WEST 45 STREET,4b1074dbf964a520d47023e3,Majestic Theatre,40.758298,-73.988154,Theater,10036,245 W 44th St,198,W 44th St
3,200 WEST 45 STREET,4a8471c9f964a520a0fc1fe3,Lunt-Fontanne Theatre,40.759090,-73.985925,Theater,10036,205 W 46th St,119,W 46th St
4,200 WEST 45 STREET,55f9eecb498e077ee156c875,Aladdin @ New Amsterdam Theatre,40.757052,-73.987488,Theater,NaN,NaN,176,NaN


In [16]:
venues.shape

(8758, 10)

In [17]:
venues.groupby(["Category"]).count()

,LocationName,Id,Name,Latitude,Longitude,PostalCode,Address,Distance,Street
Category,,,,,,,,,
Accessories Store,16,16,16,16,16,16,16,16,16
Afghan Restaurant,2,2,2,2,2,2,2,2,2
African Restaurant,14,14,14,14,14,14,14,14,14
American Restaurant,183,183,183,183,183,177,181,183,181
Animal Shelter,1,1,1,1,1,1,1,1,1
Antique Shop,9,9,9,9,9,9,9,9,9
Arcade,2,2,2,2,2,2,2,2,2
Arepa Restaurant,2,2,2,2,2,2,2,2,2
Argentinian Restaurant,2,2,2,2,2,2,2,2,2


In [18]:
venues.groupby("LocationName").size().shape

(100,)

#### Categories

Assigning categories to our location.

Based on FourSquare categories
https://developer.foursquare.com/docs/resources/categories

As we remember our clients are producers of products for women audience. Such as Jewelry, Fashion, Cosmetics etc. That's why we are trying to do some filtering for target advertising.

In [19]:
# categories
# https://developer.foursquare.com/docs/resources/categories

categories_mapping = {
    "Jewelry": ["Jewelry", "Boutique","Watch", "Accessories", "Bridal"],
    "Shoes": ["Shoe", "Boutique", "Clothing", "Accessories", "Fashion"],
    "Cosmetics": ["Cosmetics", "Spa", "Boutique", "Women"],
    "Fashion": ["Fashion", "Clothing", "Lingerie", "Art", "Theater", "Accessories"],
    "Woman_common": ["Boutique","Jewelry","Fashion","Cosmetics",
                      "Watch","Women","Bridal","Art","Theater","Spa",
                      "Clothing","Shoe","Accessories","Lingerie"],
    
}

for cat in categories_mapping:
    cat_data = venues[
        venues["Category"].str.contains("|".join(categories_mapping[cat]))
    ]
    field_counts = cat +"_count"
    field_mean_distances = cat + "_mean_distance"
    cat_counts = cat_data.groupby("LocationName")["LocationName"].count().reset_index(name=field_counts)
    cat_mean_distances = cat_data.groupby("LocationName")["Distance"].mean()\
        .round().reset_index(name=field_mean_distances)
        
    
    manhattan_kiosks = manhattan_kiosks.merge(
        cat_counts.set_index("LocationName"), 
        how="left",
        left_on="Street Address",
        right_on="LocationName")
    manhattan_kiosks = manhattan_kiosks.merge(
        cat_mean_distances.set_index("LocationName"),
        how="left",
        left_on="Street Address",
        right_on="LocationName")
    
    manhattan_kiosks[field_counts] = manhattan_kiosks[field_counts].fillna(0)
    manhattan_kiosks[field_mean_distances] = manhattan_kiosks[field_mean_distances].fillna(-1)


In [20]:
manhattan_kiosks.describe()

,Community Board,Council District,Latitude,Longitude,Smallest PPT,Postcode,Building Identification Number (BIN),Borough-Block-Lot (BBL),Census Tract (CT),Woman_common_count,Woman_common_mean_distance,Cosmetics_count,Cosmetics_mean_distance,Shoes_count,Shoes_mean_distance,Jewelry_count,Jewelry_mean_distance,Fashion_count,Fashion_mean_distance
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,106.720000,5.030000,40.775777,-73.970539,125683.600000,10024.540000,1.045557e+06,1.013151e+09,1.014032e+06,9.610000,277.820000,3.630000,254.060000,1.690000,153.670000,1.200000,97.740000,4.970000,262.150000
std,2.621126,2.302634,0.033931,0.018355,9789.796039,18.855334,2.517637e+04,5.295437e+06,6.637268e+03,9.367788,73.207658,3.304894,117.861142,3.086939,159.008929,3.357488,134.419637,7.025861,116.943168
min,101.000000,1.000000,40.715727,-74.007185,100223.000000,10001.000000,1.001642e+06,1.001500e+09,1.001600e+06,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000
25%,105.000000,3.000000,40.749868,-73.984059,120505.750000,10016.000000,1.024850e+06,1.009363e+09,1.009400e+06,4.000000,235.750000,2.000000,209.000000,0.000000,-1.000000,0.000000,-1.000000,1.000000,198.750000
50%,107.000000,4.500000,40.768754,-73.972556,122375.500000,10024.000000,1.041548e+06,1.012690e+09,1.013500e+06,7.000000,272.500000,3.000000,266.000000,1.000000,141.000000,0.000000,-1.000000,3.000000,261.000000
75%,109.000000,7.000000,40.800909,-73.954827,134597.500000,10030.250000,1.062449e+06,1.018484e+09,1.018975e+06,11.000000,330.250000,4.250000,322.000000,2.000000,290.250000,1.000000,195.500000,6.000000,337.000000
max,112.000000,10.000000,40.851165,-73.932249,145960.000000,10128.000000,1.089354e+06,1.021570e+09,1.026900e+06,45.000000,431.000000,17.000000,494.000000,16.000000,471.000000,20.000000,471.000000,42.000000,497.000000


## Methodology

Basing on exploratory data analysis we did before we can use some unsupervised learning algorithm.
As a type of machine learning algorithm used to draw inferences from datasets consisting of input data without labeled responses.

<b>What we have</b>

Unlabeled data. We don't know how to split our locations to get the best segments for targeted avertising and customers and price diversification.

<b>Our goal</b>

Get grouped information of various kiosks locations based on such features as counts of popular venues categories nearby. 

<b>Machine Learning Algorithm</b>

Due to unlabeled nature of our source data we will use some of of <b>Unsupervised Learning</b> algorithms. Without defined target labels our plan is to use <b>K-means Clustering</b>.

We can say places popularity is formed organically. As features for our data we will use this popularity rates (grouped counts of Foursquare recommended places nearby). K-mean algoritghm will help us to get:

- The centroids of the K clusters, which can be used to label our kiosks data

- Labels for the training data (each kiosk location will assigned to a single cluster)

### Clustering

In [21]:
categories_matrix = manhattan_kiosks[["Street Address"] + [x+"_count" for x in categories_mapping.keys()]]
categories_matrix.head()

,Street Address,Woman_common_count,Cosmetics_count,Shoes_count,Jewelry_count,Fashion_count
0,200 WEST 45 STREET,45.0,3.0,1.0,1.0,42.0
1,384 2 AVENUE,4.0,3.0,0.0,0.0,1.0
2,77 WEST 85 STREET,5.0,3.0,1.0,0.0,2.0
3,408 8 AVENUE,13.0,4.0,2.0,1.0,6.0
4,482 HUDSON STREET,10.0,5.0,2.0,2.0,5.0


In [22]:
# set number of clusters
kclusters = 5

grouped_clustering = categories_matrix.drop('Street Address', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)


[1 0 0 3 3 0 3 3 3 0]
(100,)


In [23]:
manhattan_kiosks["Cluster"] = kmeans.labels_

### Analysis of our clusters

In [24]:
pd.DataFrame(kmeans.cluster_centers_, columns=[x+"_count" for x in categories_mapping.keys()])

,Woman_common_count,Cosmetics_count,Shoes_count,Jewelry_count,Fashion_count
0,4.037037,2.222222,0.611111,0.203704,1.444444
1,36.500000,2.000000,1.250000,1.000000,34.250000
2,19.428571,8.571429,4.857143,2.000000,9.428571
3,10.225806,3.709677,1.290323,0.806452,5.677419
4,36.000000,15.000000,14.250000,16.500000,10.000000


Adding some comprehensive information for our clustered groups

In [25]:
clusters_info = {
    0: {
        "description": "Poor conversion rate. Low rate of related venues around. Low prices for ads on the topic",
        "color": "gray",
        "icon": "thumbs-down"
    },
    1: {
        "description": "Fashion oriented. High ads price for products about Fashion/Clothing",
        "color": "blue",
        "icon": "female"
    },
    2: {
        "description": "Meduim conversion rate. Medium average counts of all categories."\
        "Medium prices can be charged.",
        "color": "orange",
        "icon": "thumbs-up"
    },
    3: {
        "description": "Slightly normal conversion for common category for women. Low price for ads on the topic. "\
            "Medium prices for Fashion or Cosmetics ads.",
        "color": "lightgray",
        "icon": "info-sign"
    },
    4: {
        "description": "The best conversion rate. The most valuable locations."\
        "The biggest average nummbers of related venues around. "\
        "The highest price for any type of ads. Oriented to top customers.",
        "color": "red",
        "icon": "thumbs-up"
    },
    
}


Testing on sample kiosk with kmeans.predict

In [29]:
test_kiosk = categories_matrix.sample(1)
cluster_pred = kmeans.predict(
        test_kiosk.drop("Street Address", 1).values.reshape(1, -1)
    )[0]

print("Predicted Cluster: ", clusters_info[cluster_pred])

test_kiosk

Predicted Cluster:  {'description': 'Meduim conversion rate. Medium average counts of all categories.Medium prices can be charged.', 'color': 'orange', 'icon': 'thumbs-up'}


,Street Address,Woman_common_count,Cosmetics_count,Shoes_count,Jewelry_count,Fashion_count
59,197 COLUMBUS AVENUE,18.0,7.0,2.0,1.0,10.0


### Visualization

Let's see what kind of clusters we got.

See screenshots of rendered maps below the cell. Obvious way for maps presentation for GitHub problems with Folium map rendering :(

In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lon, poi, cluster in zip(
    manhattan_kiosks['Latitude'], 
    manhattan_kiosks['Longitude'], 
    manhattan_kiosks['Street Address'], 
    manhattan_kiosks['Cluster']):

    label = folium.Popup(
        str(poi) + '; Cluster: ' + clusters_info[cluster]["description"] +\
        "; Lat/Lng: "+str(lat)+","+str(lon), parse_html=True)
    
    folium.Marker(
            [lat, lon],
            popup=label,
            icon=folium.Icon(
                color=clusters_info[cluster]["color"],
                icon=clusters_info[cluster]["icon"], 
                prefix='fa'),
            
    ).add_to(map_clusters)
    
legend_html = '''
<div style="position: fixed; 
            bottom: 30px; right: 30px; width: 290px; height: 180px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background: rgba(255, 255, 255, 0.5);
            ">&nbsp; <b>Manhattan kiosks targeted ads clustering</b> <br>
              &nbsp; Best location &nbsp; <i class="fa fa-map-marker fa-2x" style="color:#d23d29"></i><br>
              &nbsp; Fashion oriented &nbsp; <i class="fa fa-map-marker fa-2x" style="color:#38a9db"></i><br>
              &nbsp; Medium &nbsp; <i class="fa fa-map-marker fa-2x" style="color:#f59630"></i><br>
              &nbsp; Slightly good &nbsp; <i class="fa fa-map-marker fa-2x" style="color:#a3a3a3"></i><br>

              &nbsp; Poor &nbsp; <i class="fa fa-map-marker fa-2x" style="color:#575757"></i>
</div>
''' 

map_clusters.get_root().html.add_child(folium.Element(legend_html))

map_clusters


### Map screenshots

Adding screenshots of Folium maps. Due to problems of their rendering on github side :(

Map with markered locations

![Map](images/map.png)

Map with popup

![Map](images/map_popup.png)

The map shows groups of clustered locations for our diversified advertising strategy

## Results and Conclusion

#### Goals

K-mean algorithm helped us to detect groups for our goals:

- Set diversified prices for different kiosks. Basing on its cluster (group, type of popularity, value)
- Targeting types of advertisement for different products. Fashion products ads can be set on kiosks from our "Fashion oriented" Cluster, Jewelry products can be targeted to "Best location" Cluster with the most valueble places, medium prices can be set for "Medium" cluster, and so on. "Poor" clusters can be targeted to differen products after additional analysis with differen target categories. If it is not popular place for women, so let's analyze it different way and send offers to new interested customers.

#### Observations on Foursquare API data

As we mentioned above during our datasets analysis we researched different methods the API provides.

According to our purposes and goals we may implement and may build different models with ideas:

- https://developer.foursquare.com/docs/api/venues/trending Returns a list of venues near the current location with the most people currently checked in. May be useful for long-term time based analysis for your strategies

https://developer.foursquare.com/docs/api/venues/suggestcompletion Returns a list of mini-venues partially matching the search term, near the location.

We could use various methods to get venues data. But our goal is to target products to popular locations that could be provided with "explore" method. That's how we can explain our choice of API endpoint.